In [17]:
import pandas as pd

In [18]:
messages=pd.read_csv("Docs/train.csv")
messages

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [19]:
messages.dropna(inplace=True)

In [20]:
messages.reset_index(inplace=True,drop=True)

In [21]:
X=messages.drop('label',axis=1)
y=messages['label']
messages=X.copy()

In [22]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

In [23]:
ps=PorterStemmer()
corpus=[]

for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review=review.lower()
    review=review.split()
    words=[ps.stem(word) for word in review if word not in stopwords.words("english")]
    word=" ".join(words)
    corpus.append(word)

In [24]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [25]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [26]:
voc_size=5000
ohe_data=[one_hot(word,voc_size) for word in corpus]

In [27]:
ohe_data

[[609, 1435, 4471, 4992, 4225, 4846, 1496, 3465, 1479, 4652],
 [103, 1398, 2663, 2905, 4867, 3651, 4268],
 [1636, 2757, 2055, 720],
 [3084, 434, 4730, 3063, 108, 2309],
 [584, 4867, 667, 3467, 1866, 2500, 4867, 4930, 629, 1591],
 [4965,
  2560,
  3413,
  1729,
  2706,
  4908,
  4055,
  671,
  1523,
  2370,
  4232,
  881,
  4592,
  2834,
  4268],
 [250, 417, 3694, 3206, 2476, 4343, 2935, 4570, 4490, 4258, 3489],
 [741, 864, 2929, 2374, 4189, 2917, 4908, 1822, 4490, 4258, 3489],
 [1165, 4523, 331, 3019, 2272, 1543, 1221, 880, 4908, 2083],
 [132, 2079, 3586, 4057, 4549, 3141, 320, 3614],
 [4865, 3845, 4068, 4780, 4414, 2587, 4006, 1720, 2762, 3048, 4579],
 [3063, 1012, 4225, 1543, 4908, 4189],
 [2108, 3155, 1223, 1055, 3189, 2174, 3786, 346, 1647],
 [355, 2242, 1880, 1678, 84, 1565, 1564, 4490, 4258, 3489],
 [197, 1329, 3850, 2918, 4909, 4490, 4258, 3489],
 [3099, 2842, 2356, 2786, 1024, 675, 4643, 2112, 427, 1257],
 [2307, 4318, 1398],
 [4305, 1374, 919, 4104, 4908, 637, 2024, 4268],
 [2

In [28]:
sent_length=30
docs=pad_sequences(ohe_data,padding="pre",maxlen=sent_length)

In [29]:
dim=40
model=Sequential()
model.add(Embedding(voc_size,dim,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 30, 40)            0         
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
import numpy as np
from sklearn.model_selection import train_test_split
X_new=np.array(docs)
y_new=np.array(y)

X_train,X_test,y_train,y_test=train_test_split(X_new,y_new,test_size=0.3,random_state=42)




In [32]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_filepath="models/news_classifier_model.h5"
model_checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64,callbacks=[model_checkpoint])

Epoch 1/20


200/200 [==============================] - ETA: 0s - loss: 0.3490 - accuracy: 0.8362
Epoch 1: val_accuracy improved from -inf to 0.90904, saving model to models\news_classifier_model.h5


c:\Users\Mohit\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


200/200 [==============================] - 23s 67ms/step - loss: 0.3490 - accuracy: 0.8362 - val_loss: 0.2066 - val_accuracy: 0.9090
Epoch 2/20
199/200 [============================>.] - ETA: 0s - loss: 0.1501 - accuracy: 0.9409
Epoch 2: val_accuracy improved from 0.90904 to 0.91178, saving model to models\news_classifier_model.h5
200/200 [==============================] - 11s 55ms/step - loss: 0.1502 - accuracy: 0.9407 - val_loss: 0.2225 - val_accuracy: 0.9118
Epoch 3/20
200/200 [==============================] - ETA: 0s - loss: 0.1078 - accuracy: 0.9623
Epoch 3: val_accuracy improved from 0.91178 to 0.91761, saving model to models\news_classifier_model.h5
200/200 [==============================] - 9s 47ms/step - loss: 0.1078 - accuracy: 0.9623 - val_loss: 0.2077 - val_accuracy: 0.9176
Epoch 4/20
199/200 [============================>.] - ETA: 0s - loss: 0.0818 - accuracy: 0.9713
Epoch 4: val_accuracy did not improve from 0.91761
200/200 [==============================] - 9s 43ms/step

In [33]:
y_pred=model.predict(X_test)

172/172 [==============================] - 3s 9ms/step


In [35]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred.round())

array([[2812,  295],
       [ 208, 2171]], dtype=int64)

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred.round())

0.9083120670798396